<a href="https://colab.research.google.com/github/CodeandoMexico/hacking-civico/blob/master/05_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#COVID-19 en México

##Carga

In [ ]:
import io
import statistics
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/LabLeón/datos_abiertos_covid19/COVID19MEXICO.csv'
data = pd.read_csv(path, encoding='latin-1')

In [ ]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-08-19,0d01b5,2,4,25,1,25,25,6,2,2020-04-13,2020-04-13,2020-04-22,2,2,60,1,2,2,1,2,2,2,1,2,1,2,2,2,99,1,99,MÃ©xico,99,2
1,2020-08-19,0392da,2,4,8,2,8,8,37,1,2020-04-30,2020-04-20,9999-99-99,97,2,40,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
2,2020-08-19,06c020,2,3,27,2,27,27,4,1,2020-04-19,2020-04-19,9999-99-99,97,2,54,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-08-19,006b91,2,4,23,1,20,23,4,2,2020-04-23,2020-04-18,9999-99-99,1,1,39,1,2,2,2,2,2,2,2,2,2,1,2,2,99,1,99,MÃ©xico,99,2
4,2020-08-19,0274c4,2,4,9,1,9,9,10,2,2020-05-04,2020-05-03,9999-99-99,2,1,49,1,2,2,2,2,2,2,1,2,2,1,2,2,99,1,99,MÃ©xico,99,2


In [ ]:
path_dic = '/content/drive/My Drive/LabLeón/diccionario_datos_covid19/Descriptores_0419.xlsx'
diccionario = pd.read_excel(path_dic)

In [ ]:
diccionario

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


In [ ]:
path_cat = '/content/drive/My Drive/LabLeón/diccionario_datos_covid19/Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(path_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

In [ ]:
catalogues

{'ENTIDADES':     clave_entidad               entidad_federativa abreviatura
 0               1                   AGUASCALIENTES          AS
 1               2                  BAJA CALIFORNIA          BC
 2               3              BAJA CALIFORNIA SUR          BS
 3               4                         CAMPECHE          CC
 4               5             COAHUILA DE ZARAGOZA          CL
 5               6                           COLIMA          CM
 6               7                          CHIAPAS          CS
 7               8                        CHIHUAHUA          CH
 8               9                 CIUDAD DE MÉXICO          DF
 9              10                          DURANGO          DG
 10             11                       GUANAJUATO          GT
 11             12                         GUERRERO          GR
 12             13                          HIDALGO          HG
 13             14                          JALISCO          JC
 14             15         

In [ ]:
catalogues['SEXO']

,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


##Perfilamiento

###Descripción General

In [ ]:
# dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1211552
Número de columnas: 35


In [ ]:
# columnas (variables)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [ ]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [ ]:
# calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

In [ ]:
# tipos de datos
data.dtypes

fecha_actualizacion    object
id_registro            object
origen                  int64
sector                  int64
entidad_um              int64
sexo                    int64
entidad_nac             int64
entidad_res             int64
municipio_res           int64
tipo_paciente           int64
fecha_ingreso          object
fecha_sintomas         object
fecha_def              object
intubado                int64
neumonia                int64
edad                    int64
nacionalidad            int64
embarazo                int64
habla_lengua_indig      int64
diabetes                int64
epoc                    int64
asma                    int64
inmusupr                int64
hipertension            int64
otra_com                int64
cardiovascular          int64
obesidad                int64
renal_cronica           int64
tabaquismo              int64
otro_caso               int64
resultado               int64
migrante                int64
pais_nacionalidad      object
pais_orige

###Limpieza

In [ ]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [ ]:
data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-08-19,0d01b5,2,4,25,1,25,25,6,2,2020-04-13,2020-04-13,2020-04-22,2,2,60,1,2,2,1,2,2,2,1,2,1,2,2,2,99,1,99,MÃ©xico,99,2
1,2020-08-19,0392da,2,4,8,2,8,8,37,1,2020-04-30,2020-04-20,NaT,97,2,40,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
2,2020-08-19,06c020,2,3,27,2,27,27,4,1,2020-04-19,2020-04-19,NaT,97,2,54,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-08-19,006b91,2,4,23,1,20,23,4,2,2020-04-23,2020-04-18,NaT,1,1,39,1,2,2,2,2,2,2,2,2,2,1,2,2,99,1,99,MÃ©xico,99,2
4,2020-08-19,0274c4,2,4,9,1,9,9,10,2,2020-05-04,2020-05-03,NaT,2,1,49,1,2,2,2,2,2,2,1,2,2,1,2,2,99,1,99,MÃ©xico,99,2


In [ ]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [ ]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
  data = (
      data.merge(catalogues['ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
      drop(columns=[col, 'abreviatura', 'clave_entidad']).
      rename(columns={'entidad_federativa':col})
  )

In [ ]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
  data = (
      data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
      drop(columns=[col, 'clave']).
      rename(columns={'descripción':col})
  )

In [ ]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
  data = (
      data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
      drop(columns=[col, 'clave']).
      rename(columns={'descripción':col})
  )

In [ ]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [ ]:
data.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-19,0d01b5,2020-04-13,2020-04-13,2020-04-22,60,MÃ©xico,99,SINALOA,CULIACÁN,SINALOA,SINALOA,NO,NO,NO,NO,SI,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
1,2020-08-19,0392da,2020-04-30,2020-04-20,NaT,40,MÃ©xico,99,CHIHUAHUA,JUÁREZ,CHIHUAHUA,CHIHUAHUA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
2,2020-08-19,06c020,2020-04-19,2020-04-19,NaT,54,MÃ©xico,99,TABASCO,CENTRO,TABASCO,TABASCO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
3,2020-08-19,006b91,2020-04-23,2020-04-18,NaT,39,MÃ©xico,99,QUINTANA ROO,OTHÓN P. BLANCO,QUINTANA ROO,OAXACA,SI,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
4,2020-08-19,0274c4,2020-05-04,2020-05-03,NaT,49,MÃ©xico,99,CIUDAD DE MÉXICO,ÁLVARO OBREGÓN,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


###Filtros

**Distintas formas de filtrar lo mismo**

In [ ]:
# usando query
data.query('nacionalidad == "MEXICANA"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-19,0d01b5,2020-04-13,2020-04-13,2020-04-22,60,MÃ©xico,99,SINALOA,CULIACÁN,SINALOA,SINALOA,NO,NO,NO,NO,SI,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
1,2020-08-19,0392da,2020-04-30,2020-04-20,NaT,40,MÃ©xico,99,CHIHUAHUA,JUÁREZ,CHIHUAHUA,CHIHUAHUA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020


In [ ]:
# haciendo referencia a la columna como índice
data[data['nacionalidad'] == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-19,0d01b5,2020-04-13,2020-04-13,2020-04-22,60,MÃ©xico,99,SINALOA,CULIACÁN,SINALOA,SINALOA,NO,NO,NO,NO,SI,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
1,2020-08-19,0392da,2020-04-30,2020-04-20,NaT,40,MÃ©xico,99,CHIHUAHUA,JUÁREZ,CHIHUAHUA,CHIHUAHUA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020


In [ ]:
# seleccionando directamente la columna que queremos filtrar
data[data.nacionalidad == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-08-19,0d01b5,2020-04-13,2020-04-13,2020-04-22,60,MÃ©xico,99,SINALOA,CULIACÁN,SINALOA,SINALOA,NO,NO,NO,NO,SI,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
1,2020-08-19,0392da,2020-04-30,2020-04-20,NaT,40,MÃ©xico,99,CHIHUAHUA,JUÁREZ,CHIHUAHUA,CHIHUAHUA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020


**Filtrado múltiple**

In [ ]:
# usando query podemos poner en el string del query todas las condiciones que queramos
data.query('nacionalidad == "MEXICANA" & entidad_res == "CIUDAD DE MÉXICO"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
4,2020-08-19,0274c4,2020-05-04,2020-05-03,NaT,49,MÃ©xico,99,CIUDAD DE MÉXICO,ÁLVARO OBREGÓN,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020
6,2020-08-19,126f5c,2020-05-02,2020-05-02,NaT,28,MÃ©xico,99,CIUDAD DE MÉXICO,IZTACALCO,MÉXICO,CIUDAD DE MÉXICO,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


In [ ]:
# usando el filtrado tradicional debemos encerrar en paréntesis cada condición que queremos que cumpla nuestro filtro
data[(data['nacionalidad'] == "MEXICANA") & (data.entidad_res == "CIUDAD DE MÉXICO")].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
4,2020-08-19,0274c4,2020-05-04,2020-05-03,NaT,49,MÃ©xico,99,CIUDAD DE MÉXICO,ÁLVARO OBREGÓN,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020
6,2020-08-19,126f5c,2020-05-02,2020-05-02,NaT,28,MÃ©xico,99,CIUDAD DE MÉXICO,IZTACALCO,MÉXICO,CIUDAD DE MÉXICO,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


###Resúmenes

In [ ]:
# una columna, una métrica
data.groupby('sexo').edad.mean()

sexo
HOMBRE    42.982222
MUJER     41.842180
Name: edad, dtype: float64

In [ ]:
# una columna, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max']})

edad                
        min       mean  max
sexo                       
HOMBRE    0  42.982222  120
MUJER     0  41.842180  120

In [ ]:
# varias columnas, una métrica
data.groupby('sexo').agg({'edad': 'mean', 'mes_ingreso': 'mean', 'mes_def': 'mean'})

,edad,mes_ingreso,mes_def
sexo,,,
HOMBRE,42.982222,6.312419,6.178929
MUJER,41.842180,6.365031,6.238480


In [ ]:
# varias columnas, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max'], 'mes_ingreso': ['min','mean','max'], 'mes_def': ['min','mean','max']})

edad                 mes_ingreso               mes_def               
        min       mean  max         min      mean max     min      mean  max
sexo                                                                        
HOMBRE    0  42.982222  120           1  6.312419   8     1.0  6.178929  8.0
MUJER     0  41.842180  120           1  6.365031   8     1.0  6.238480  8.0

###Descripción por Tipo

In [ ]:
# Filtrar por tipos generales (categóricas y numéricas)
numerical = data.select_dtypes(include='number')
categorical = data.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])

In [ ]:
numerical.head()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
0,60,4.0,2020.0,4,2020
1,40,NaN,NaN,4,2020
2,54,NaN,NaN,4,2020
3,39,NaN,NaN,4,2020
4,49,NaN,NaN,5,2020


In [ ]:
categorical.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion
0,2020-08-19,0d01b5,2020-04-13,2020-04-13,2020-04-22,MÃ©xico,99,SINALOA,CULIACÁN,SINALOA,SINALOA,NO,NO,NO,NO,SI,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True
1,2020-08-19,0392da,2020-04-30,2020-04-20,NaT,MÃ©xico,99,CHIHUAHUA,JUÁREZ,CHIHUAHUA,CHIHUAHUA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
2,2020-08-19,06c020,2020-04-19,2020-04-19,NaT,MÃ©xico,99,TABASCO,CENTRO,TABASCO,TABASCO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
3,2020-08-19,006b91,2020-04-23,2020-04-18,NaT,MÃ©xico,99,QUINTANA ROO,OTHÓN P. BLANCO,QUINTANA ROO,OAXACA,SI,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False
4,2020-08-19,0274c4,2020-05-04,2020-05-03,NaT,MÃ©xico,99,CIUDAD DE MÉXICO,ÁLVARO OBREGÓN,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO,NO,NO,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False


In [ ]:
# para conocer todos los tipo de datos
?data.select_dtypes

####Categóricas

In [ ]:
# Cardinalidad - Valores únicos por columna
for col in categorical.columns:
  print('Valores únicos en "{}": {}'.format(col,categorical[col].nunique()))

Valores únicos en "fecha_actualizacion": 1
Valores únicos en "id_registro": 1211552
Valores únicos en "fecha_ingreso": 232
Valores únicos en "fecha_sintomas": 232
Valores únicos en "fecha_def": 177
Valores únicos en "pais_nacionalidad": 114
Valores únicos en "pais_origen": 72
Valores únicos en "entidad_res": 32
Valores únicos en "municipio_res": 2133
Valores únicos en "entidad_um": 32
Valores únicos en "entidad_nac": 33
Valores únicos en "intubado": 4
Valores únicos en "neumonia": 3
Valores únicos en "embarazo": 4
Valores únicos en "habla_lengua_indig": 3
Valores únicos en "diabetes": 3
Valores únicos en "epoc": 3
Valores únicos en "asma": 3
Valores únicos en "inmusupr": 3
Valores únicos en "hipertension": 3
Valores únicos en "otra_com": 3
Valores únicos en "cardiovascular": 3
Valores únicos en "obesidad": 3
Valores únicos en "renal_cronica": 3
Valores únicos en "tabaquismo": 3
Valores únicos en "otro_caso": 3
Valores únicos en "migrante": 3
Valores únicos en "uci": 4
Valores únicos en

In [ ]:
# Unicidad - proporción de valores únicos por columna
total = data.shape[0]
for col in categorical.columns:
  print('Porcentaje de valores únicos en "{}": {}%'.format(col,round(categorical[col].nunique()/total*100,2)))

Porcentaje de valores únicos en "fecha_actualizacion": 0.0%
Porcentaje de valores únicos en "id_registro": 100.0%
Porcentaje de valores únicos en "fecha_ingreso": 0.02%
Porcentaje de valores únicos en "fecha_sintomas": 0.02%
Porcentaje de valores únicos en "fecha_def": 0.01%
Porcentaje de valores únicos en "pais_nacionalidad": 0.01%
Porcentaje de valores únicos en "pais_origen": 0.01%
Porcentaje de valores únicos en "entidad_res": 0.0%
Porcentaje de valores únicos en "municipio_res": 0.18%
Porcentaje de valores únicos en "entidad_um": 0.0%
Porcentaje de valores únicos en "entidad_nac": 0.0%
Porcentaje de valores únicos en "intubado": 0.0%
Porcentaje de valores únicos en "neumonia": 0.0%
Porcentaje de valores únicos en "embarazo": 0.0%
Porcentaje de valores únicos en "habla_lengua_indig": 0.0%
Porcentaje de valores únicos en "diabetes": 0.0%
Porcentaje de valores únicos en "epoc": 0.0%
Porcentaje de valores únicos en "asma": 0.0%
Porcentaje de valores únicos en "inmusupr": 0.0%
Porcenta

In [ ]:
# Moda - valor que más se repite en cada columna excepto los indicadores únicos
for col in categorical.drop(columns='id_registro').columns:
  print('Valor más popular en "{}": {}'.format(col, statistics.mode(categorical[col])))

Valor más popular en "fecha_actualizacion": 2020-08-19 00:00:00
Valor más popular en "fecha_ingreso": 2020-07-20 00:00:00
Valor más popular en "fecha_sintomas": 2020-07-20 00:00:00
Valor más popular en "fecha_def": NaT
Valor más popular en "pais_nacionalidad": MÃ©xico
Valor más popular en "pais_origen": 99
Valor más popular en "entidad_res": CIUDAD DE MÉXICO
Valor más popular en "municipio_res": IZTAPALAPA
Valor más popular en "entidad_um": CIUDAD DE MÉXICO
Valor más popular en "entidad_nac": CIUDAD DE MÉXICO
Valor más popular en "intubado": NO APLICA
Valor más popular en "neumonia": NO 
Valor más popular en "embarazo": NO APLICA
Valor más popular en "habla_lengua_indig": NO 
Valor más popular en "diabetes": NO 
Valor más popular en "epoc": NO 
Valor más popular en "asma": NO 
Valor más popular en "inmusupr": NO 
Valor más popular en "hipertension": NO 
Valor más popular en "otra_com": NO 
Valor más popular en "cardiovascular": NO 
Valor más popular en "obesidad": NO 
Valor más popular

####Numéricas

In [ ]:
numerical.describe()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
count,1.211552e+06,74783.000000,74783.0,1.211552e+06,1211552.0
mean,4.240750e+01,6.200513,2020.0,6.338942e+00,2020.0
std,1.671605e+01,1.144832,0.0,1.203029e+00,0.0
min,0.000000e+00,1.000000,2020.0,1.000000e+00,2020.0
25%,3.000000e+01,5.000000,2020.0,6.000000e+00,2020.0
50%,4.100000e+01,6.000000,2020.0,7.000000e+00,2020.0
75%,5.300000e+01,7.000000,2020.0,7.000000e+00,2020.0
max,1.200000e+02,8.000000,2020.0,8.000000e+00,2020.0


##Análisis Exploratorio

En esta sección responderemos algunas preguntas sobre el set de datos usando las herramientas que hemos visto hasta ahora

In [ ]:
# ¿Cuántas pruebas por origen se realizan?
data.groupby('origen').id_registro.count()

origen
FUERA DE USMER    803255
USMER             408297
Name: id_registro, dtype: int64

In [ ]:
data.origen.value_counts()

FUERA DE USMER    803255
USMER             408297
Name: origen, dtype: int64

In [ ]:
# ¿Cuántas pruebas por sector se realizan?
data.sector.value_counts()

SSA                745128
IMSS               316118
ISSSTE              46310
PRIVADA             46058
ESTATAL             23444
PEMEX               11026
IMSS-BIENESTAR       8422
SEMAR                6687
SEDENA               6014
MUNICIPAL            1034
UNIVERSITARIO         797
DIF                   326
CRUZ ROJA             180
NO ESPECIFICADO         8
Name: sector, dtype: int64

In [ ]:
# ¿Cuántos pacientes por sexo tenemos?
data.sexo.value_counts()

MUJER     610770
HOMBRE    600782
Name: sexo, dtype: int64

In [ ]:
# ¿Cuál es la proporción de los pacientes por sexo?
data.sexo.value_counts(normalize=True)

MUJER     0.504122
HOMBRE    0.495878
Name: sexo, dtype: float64

In [ ]:
# ¿Cuál es la proporción de resultados de las pruebas?
data.resultado.value_counts(normalize=True)

No positivo SARS-CoV-2    0.488330
Positivo SARS-CoV-2       0.443259
Resultado pendiente       0.068411
Name: resultado, dtype: float64

In [ ]:
# ¿Cuántas pruebas positivas tenemos en el set?
data[data.resultado == 'Positivo SARS-CoV-2'].id_registro.count()

537031

In [ ]:
# ¿Cuántos casos positivos tenemos por mes?
data[data.resultado == 'Positivo SARS-CoV-2'].mes_ingreso.value_counts().sort_index()

1         2
2         7
3      2521
4     26409
5     85953
6    152145
7    196676
8     73318
Name: mes_ingreso, dtype: int64

In [ ]:
# ¿Cuál es el resultado de las pruebas por sexo?
data.groupby(['sexo','resultado'])['id_registro'].count()

sexo    resultado             
HOMBRE  No positivo SARS-CoV-2    276203
        Positivo SARS-CoV-2       283145
        Resultado pendiente        41434
MUJER   No positivo SARS-CoV-2    315434
        Positivo SARS-CoV-2       253886
        Resultado pendiente        41450
Name: id_registro, dtype: int64

In [ ]:
data.groupby('sexo').resultado.value_counts()

sexo    resultado             
HOMBRE  Positivo SARS-CoV-2       283145
        No positivo SARS-CoV-2    276203
        Resultado pendiente        41434
MUJER   No positivo SARS-CoV-2    315434
        Positivo SARS-CoV-2       253886
        Resultado pendiente        41450
Name: resultado, dtype: int64

In [ ]:
# ¿Qué proporción de resultados de pruebas tenemos por sexo?
data.groupby('sexo').resultado.value_counts(normalize=True)

sexo    resultado             
HOMBRE  Positivo SARS-CoV-2       0.471294
        No positivo SARS-CoV-2    0.459739
        Resultado pendiente       0.068967
MUJER   No positivo SARS-CoV-2    0.516453
        Positivo SARS-CoV-2       0.415682
        Resultado pendiente       0.067865
Name: resultado, dtype: float64

In [ ]:
# ¿Cuál es la edad de los pacientes por sexo?
data.groupby('sexo').edad.describe()

,count,mean,std,min,25%,50%,75%,max
sexo,,,,,,,,
HOMBRE,600782.0,42.982222,17.014159,0.0,31.0,42.0,54.0,120.0
MUJER,610770.0,41.842180,16.397910,0.0,30.0,40.0,52.0,120.0


In [ ]:
def q25(x):
  return x.quantile(0.25)

def q75(x):
  return x.quantile(0.75)

data.groupby('sexo').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                    
        min q25 median q75  max
sexo                           
HOMBRE    0  31     42  54  120
MUJER     0  30     40  52  120

In [ ]:
# ¿Cuál es la edad de los pacientes dependiendo de si son intubados o no?
data.groupby('intubado').edad.describe()

,count,mean,std,min,25%,50%,75%,max
intubado,,,,,,,,
NO,204355.0,53.688513,19.248945,0.0,42.0,55.0,67.0,120.0
NO APLICA,987392.0,39.817834,14.916593,0.0,29.0,38.0,49.0,120.0
NO ESPECIFICADO,157.0,52.394904,17.951058,0.0,41.0,53.0,65.0,92.0
SI,19648.0,55.137419,19.202082,0.0,46.0,58.0,68.0,108.0


In [ ]:
data.groupby('intubado').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                    
                 min q25 median q75  max
intubado                                
NO                 0  42     55  67  120
NO APLICA          0  29     38  49  120
NO ESPECIFICADO    0  41     53  65   92
SI                 0  46     58  68  108

In [ ]:
# ¿Cuál es la edad máxima y cuál es la mínima de los pacientes fallecidos?
edad_def_max = data[data.defuncion].edad.max()
data[data.defuncion & (data.edad == edad_def_max)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
592516,2020-08-19,1b7b20,2020-05-21,2020-05-20,2020-05-26,109,MÃ©xico,99,MICHOACÁN DE OCAMPO,APATZINGÁN,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,NO,SI,NO APLICA,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,5.0,2020.0,5,2020


In [ ]:
edad_def_min = data[data.defuncion].edad.min()
data[data.defuncion & (data.edad == edad_def_min)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
11953,2020-08-19,083c34,2020-05-25,2020-05-25,2020-06-01,0,MÃ©xico,99,MÉXICO,CHIMALHUACÁN,MÉXICO,MÉXICO,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,6.0,2020.0,5,2020
15330,2020-08-19,02f8e2,2020-05-08,2020-05-01,2020-05-10,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
37041,2020-08-19,053398,2020-06-11,2020-06-11,2020-06-17,0,MÃ©xico,99,AGUASCALIENTES,AGUASCALIENTES,AGUASCALIENTES,AGUASCALIENTES,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,6.0,2020.0,6,2020
43920,2020-08-19,014b5d,2020-05-18,2020-05-17,2020-05-18,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
53245,2020-08-19,1b82ee,2020-05-13,2020-05-13,2020-05-22,0,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,CIUDAD DE MÉXICO,MÉXICO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143249,2020-08-19,082242,2020-08-07,2020-08-06,2020-08-10,0,MÃ©xico,99,ZACATECAS,ZACATECAS,ZACATECAS,ZACATECAS,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,8.0,2020.0,8,2020
1148939,2020-08-19,0860f4,2020-06-30,2020-06-30,2020-07-13,0,MÃ©xico,99,PUEBLA,TEHUACÁN,PUEBLA,PUEBLA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,7.0,2020.0,6,2020
1164945,2020-08-19,0eb585,2020-08-15,2020-08-15,2020-08-17,0,MÃ©xico,99,TAMAULIPAS,REYNOSA,TAMAULIPAS,TAMAULIPAS,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,8.0,2020.0,8,2020
1173598,2020-08-19,05caa0,2020-08-17,2020-08-15,2020-08-17,0,MÃ©xico,99,MICHOACÁN DE OCAMPO,MORELIA,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,NO,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,8.0,2020.0,8,2020


In [ ]:
# ¿Cuál es la edad de los pacientes que fallecen?
data.groupby('defuncion').edad.describe()

,count,mean,std,min,25%,50%,75%,max
defuncion,,,,,,,,
False,1136769.0,41.141837,16.018981,0.0,30.0,40.0,51.0,120.0
True,74783.0,61.646738,15.224774,0.0,52.0,63.0,72.0,109.0


In [ ]:
# ¿Cuántas defunciones tenemos por mes?
data[data.defuncion].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      232
4.0     5237
5.0    15924
6.0    20980
7.0    22617
8.0     9775
Name: mes_def, dtype: int64

In [ ]:
# ¿Cuál es la proporción de diábeticos para las defunciones?
data.groupby('diabetes').defuncion.value_counts(normalize=True)

diabetes   defuncion
NO         False        0.956465
           True         0.043535
SE IGNORA  False        0.873029
           True         0.126971
SI         False        0.811509
           True         0.188491
Name: defuncion, dtype: float64

In [ ]:
# ¿Cuál es la proporción de asmáticos para las defunciones?
data.groupby('asma').defuncion.value_counts(normalize=True)

asma       defuncion
NO         False        0.937903
           True         0.062097
SE IGNORA  False        0.862739
           True         0.137261
SI         False        0.957227
           True         0.042773
Name: defuncion, dtype: float64

# Ejercicio

Contesta a las siguientes preguntas

In [ ]:
# ¿Tienen los pacientes con hipertensión un riesgo más alto de defunción?

In [ ]:
# ¿Cuántos casos confirmados se tienen por Estado?

In [ ]:
# ¿Cuántas defunciones se tienen por Estado?

In [ ]:
# ¿Cuántos fallecimientos han ocurrido en el Estado con mayor número de casos confirmados?

In [ ]:
# ¿Cuántos fallecimientos han ocurrido en los pacientes Ambulatorios?